In [3]:
import selenium
from Scraping import *

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/Rgao/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [4]:
driver.get("http://www.randhawa.us/games/retailer/nyu.html")

# Define Strategy

In [11]:
# import gurobi
# from gurobi import *
# prob = Model('Optimization')
# x9 = prob.addVar(vtype=GRB.INTEGER,lb = 0, name = 'x_.9')
# x8= prob.addVar(lb = 0, name = 'x_.8')
# x6 = prob.addVar(lb = 0, name = 'x_.6')
# xM = prob.addVar(lb = 0, name = 'x_M')
# a=70
# p=60
# prob.addConstr(xM>=1,name="C1")
# prob.addConstr(a*(x9+x8+x6+xM)<= 2000,name="C2")
# prob.addConstr(x9+x8+x6+xM <= 15,name="C3")
# obj = a*x9*p*.9 + a*x8*p*.8 + a*x6*p*.6 + a*xM*p
# prob.setObjective(obj, GRB.MAXIMIZE)




In [12]:
table

,Week,Price,Sales,Remaining,foresight,Choice,diff,revenue,perfect
0,1,60,87,1913,"Your revenue: $83,442, Perfect foresight strat...",0,10.3,83442.0,93072.0
1,2,60,101,1812,"Your revenue: $83,442, Perfect foresight strat...",0,10.3,83442.0,93072.0
2,3,60,76,1736,"Your revenue: $83,442, Perfect foresight strat...",0,10.3,83442.0,93072.0
3,4,54,84,1652,"Your revenue: $83,442, Perfect foresight strat...",1,10.3,83442.0,93072.0
4,5,54,75,1577,"Your revenue: $83,442, Perfect foresight strat...",0,10.3,83442.0,93072.0
5,6,48,109,1468,"Your revenue: $83,442, Perfect foresight strat...",2,10.3,83442.0,93072.0
6,7,48,78,1390,"Your revenue: $83,442, Perfect foresight strat...",0,10.3,83442.0,93072.0
7,8,36,160,1230,"Your revenue: $83,442, Perfect foresight strat...",3,10.3,83442.0,93072.0
8,9,36,138,1092,"Your revenue: $83,442, Perfect foresight strat...",0,10.3,83442.0,93072.0
9,10,36,162,930,"Your revenue: $83,442, Perfect foresight strat...",0,10.3,83442.0,93072.0


In [27]:
def strategy(table,choices,chosenchoices):
    
    week=max(table['Week'].astype(int))
    if(week > 1):
        prev_choice=chosenchoices[week-1]
        lift=(int(table['Sales'][week-1])-int(table['Sales'][week-2]))/int(table['Sales'][week-2])
        lastchoice=max(chosenchoices)
        remainingweeks=15-week
        canfinish=False
        remaining=table['Remaining'][week-1]
        if(remainingweeks*table['Sales'][week-1]>remaining):
            canfinish=True
        
        if(lift<0 and canfinish == False):
            print("negative lift: "+ str(lift))
            return(min(lastchoice+1,3))
    return(0)

#strategy(table,[0,1,2,3])
#table

### Get Buttons

In [28]:
html_source = driver.page_source

# Controls
restart_game = driver.find_element_by_id('practiceButton')
maintain = driver.find_element_by_id('maintainButton')
ten = driver.find_element_by_id('tenButton')
twenty = driver.find_element_by_id('twentyButton')
forty = driver.find_element_by_id('fortyButton')



def main_game_click(click_type=0):
#     global click_count
#     if click_count > 15:
#         print('Error: cannot go over 15 weeks.')
#         return
    if click_type == 0:
        maintain.click()
    if click_type == 1:
        ten.click()
    if click_type ==2:
        twenty.click()
    if click_type == 3:
        forty.click()
#     click_count += 1


def get_results(results_df, performance_df, game_id):
    table = driver.find_elements_by_id("result-table")
    for row in table:
        for cell_number in range(1, 16):
            cell = row.find_elements_by_tag_name("tr")[cell_number]
            print(cell.text)
            results_list = cell.text.split(' ')
            print(results_list)
            results_df = results_df.append({'Round': game_id, 'Week': results_list[0], 'Price': results_list[1], 'Sales': results_list[2],
                               'Remaining': results_list[3]}, ignore_index=True)

    revenue = driver.find_element_by_id("rev")
    perfect = driver.find_element_by_id("perfect")
    difference = driver.find_element_by_id("percentage")
    print(f'Your revenue is: {revenue.text}')
    print(f'Perfect revenue is: {perfect.text}')
    print(f'Difference is: {difference.text}')
    performance_df = performance_df.append({'Round': game_id, 'Revenue': revenue.text, 'Perfect': perfect.text, 'Difference': difference.text.split(' ')[2]}, ignore_index=True)

    return results_df, performance_df

# Try out controls
'''
main_game_click()
main_game_click()
main_game_click()
main_game_click()
main_game_click('ten')
main_game_click()
main_game_click()
main_game_click()
main_game_click('twenty')
main_game_click()
main_game_click()
main_game_click()
main_game_click('forty')
'''

results_df = pd.DataFrame(columns=['Round', 'Week', 'Price', 'Sales', 'Remaining'])
performance_df = pd.DataFrame(columns=['Round', 'Revenue', 'Perfect', 'Difference'])


In [29]:
all_data=[]
foresights=[]
driver.find_elements_by_xpath("//input[contains(@id,'practiceButton')]")[0].click()
#choices=[[0]*15,[1]+[0]*14,[2]+[0]*14,[3]+[0]*14] # These are the strategies to scrape
#choices=[choices[0]]
attempts=10

for choicerange in range(attempts):#choices:
    chosenchoices=[0]
    driver.find_elements_by_xpath("//input[contains(@id,'practiceButton')]")[0].click()
    for i in range(0,15):
        #hoice=choices[random.randint(0,len(choices)-1)]

        data=driver.find_elements_by_xpath("//table[contains(@id,'result-table')]")[0].text
        splitdata=data.split("\n")
        columns=splitdata[0].split(" ")
        values_split=[]
        for values in splitdata[1:len(splitdata)]:
            values_split.append(values.split(" "))

        table=pd.DataFrame(values_split)
        table.columns=columns[0:4]
        
        choice=strategy(table,[0,1,2,3],chosenchoices)#choicerange[i]
        chosenchoices.append(choice)
        main_game_click(choice)
        #click
#         maintain=driver.find_element_by_xpath("//input[contains(@id,'maintainButton')]")
#         discount10=driver.find_element_by_xpath("//input[contains(@id,'tenButton')]")
#         discount20=driver.find_element_by_xpath("//input[contains(@id,'twentyButton')]")
#         discount40=driver.find_element_by_xpath("//input[contains(@id,'fortyButton')]")
#         buttonstoclick=[maintain,discount10,discount20,discount40]
#         buttonstoclick[choice].click()
        if(choice==3):
            data=driver.find_elements_by_xpath("//table[contains(@id,'result-table')]")[0].text
            splitdata=data.split("\n")
            columns=splitdata[0].split(" ")
            values_split=[]
            for values in splitdata[1:len(splitdata)]:
                values_split.append(values.split(" "))

            table=pd.DataFrame(values_split)
            table.columns=columns[0:4]
            chosenchoices.extend([0]*(13-i))
            break
    foresight=driver.find_element_by_xpath("//div[contains(@style,'position:relative;top:10px;left:6px;')]").text
    table['foresight']=foresight
    table['Choice']=chosenchoices
    table['diff']=float(table['foresight'][0].split(": ")[3].replace("%",""))
    table['revenue']=float(table['foresight'][0].split(", ")[0].split(": ")[1].replace("$","").replace(",",""))
    table['perfect']=float(table['foresight'][0].split(", ")[1].split(": ")[1].replace("$","").replace(",",""))
    
    all_data.append(table)
# print(table)

negative lift: -0.037037037037037035
negative lift: -0.28
negative lift: -0.1323529411764706
negative lift: -0.09375


ValueError: Length of values (16) does not match length of index (15)

In [30]:
chosenchoices

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [29]:
all_data=pd.concat(all_data).reset_index(drop=True)
all_data

,Week,Price,Sales,Remaining,foresight,Choice,diff,revenue,perfect
0,1,60,120,1880,"Your revenue: $84,702, Perfect foresight strat...",0,25.1,84702.0,113082.0
1,2,60,113,1767,"Your revenue: $84,702, Perfect foresight strat...",0,25.1,84702.0,113082.0
2,3,54,105,1662,"Your revenue: $84,702, Perfect foresight strat...",1,25.1,84702.0,113082.0
3,4,48,243,1419,"Your revenue: $84,702, Perfect foresight strat...",2,25.1,84702.0,113082.0
4,5,48,192,1227,"Your revenue: $84,702, Perfect foresight strat...",0,25.1,84702.0,113082.0
...,...,...,...,...,...,...,...,...,...
145,11,36,197,849,"Your revenue: $82,608, Perfect foresight strat...",3,6.5,82608.0,88320.0
146,12,36,162,687,"Your revenue: $82,608, Perfect foresight strat...",0,6.5,82608.0,88320.0
147,13,36,146,541,"Your revenue: $82,608, Perfect foresight strat...",0,6.5,82608.0,88320.0
148,14,36,206,335,"Your revenue: $82,608, Perfect foresight strat...",0,6.5,82608.0,88320.0


In [203]:
pd.concat(all_data).to_csv("games.csv")